In [1]:
import os
from dotenv import load_dotenv

# --- LangChain & Google Gemini Imports ---
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load environment variables from .env file
load_dotenv()

True

In [4]:
# --- Configuration & Model Initialization ---
FAISS_INDEX_PATH = "faiss_store_finfriend_gemini"
DOC_DIRECTORY = "./documents"

if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY not found. Please set it in your .env file.")

print("Initializing models...")
# Initialize the FREE, local embedding model
embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

# Initialize the language model using the Google Gemini API
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)
print("Models initialized successfully! ✅")

Initializing models...
Models initialized successfully! ✅


In [5]:
# --- Ingestion Process ---
print(f"Processing documents from '{DOC_DIRECTORY}'...")
loader = DirectoryLoader(
    DOC_DIRECTORY, 
    glob="**/*.txt", 
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Split documents into {len(chunks)} chunks.")

vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding_model)
vectorstore.save_local(FAISS_INDEX_PATH)
print(f"Index created and saved to '{FAISS_INDEX_PATH}'! ✅")

Processing documents from './documents'...
Split documents into 112 chunks.
Index created and saved to 'faiss_store_finfriend_gemini'! ✅


In [6]:
# Your full, detailed prompt instructions
FIN_FRIEND_PROMPT_INSTRUCTIONS = """
### Persona
You are Fin-Friend, an expert, empathetic, and encouraging financial guide in India...
(Your full, detailed prompt goes here)
...
**Mandatory Disclaimer:** End with the required disclaimer.
"""

def format_user_data_for_llm(data):
    # (Your existing smart formatting function)
    report_lines = []
    for key, value in data.items():
        title = key.replace('_', ' ').title()
        if isinstance(value, dict):
            report_lines.append(f"**{title}:**")
            for sub_key, sub_value in value.items():
                report_lines.append(f"- {sub_key.replace('_', ' ').title()}: {sub_value}")
        else:
            report_lines.append(f"**{title}:**\n{value}")
        report_lines.append("")
    return "\n".join(report_lines)

def get_user_input(prompt_text):
    return input(prompt_text + "\n> ")

In [9]:
# --- Phase 1: Structured Data Gathering ---
user_data = {}
expenses = {}
print("Hello! I'm Fin-Friend... Let's get started!\n")
# (Your structured data gathering code here...)
user_data['income_salary'] = get_user_input("First, what is your fixed monthly take-home salary?")
user_data['income_other'] = get_user_input("Do you have any other sources of income?")
print("\nGreat. Now let's break down your monthly expenses.")
expenses['rent_or_emi'] = get_user_input("- Rent or Home Loan EMI:")
expenses['utilities'] = get_user_input("- Electricity, Water, Gas:")
user_data['expenses_structured'] = expenses
user_data['financial_goals'] = get_user_input("\nWhat are your major financial goals?")
user_data['current_investments'] = get_user_input("\nBriefly, what investments do you have?")
user_data['outstanding_debts'] = get_user_input("\nBriefly, what outstanding debts do you have?")

print("\nThank you for the information. Generating your personalized financial health report...")

# --- Phase 2: RAG-Powered Analysis for the Initial Report ---
vectorstore = FAISS.load_local(FAISS_INDEX_PATH, embedding_model, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
formatted_user_data = format_user_data_for_llm(user_data)

# Dynamic Query Generation
query_generation_prompt_template = "Based on the user's financial data below, generate a short, specific question to search a financial guide for relevant advice. Focus on the most critical area.\n\nUSER DATA:\n{user_data}\n\nSEARCH QUERY:"
query_generation_prompt = ChatPromptTemplate.from_template(query_generation_prompt_template)
query_generation_chain = query_generation_prompt | llm | StrOutputParser()
dynamic_query = query_generation_chain.invoke({"user_data": formatted_user_data})

# Dynamic Retrieval
retrieved_docs = retriever.invoke(dynamic_query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

# Report Generation
final_prompt_template = """{instructions}\n\n---
**CONTEXT FROM FINANCIAL GUIDE:**
{context}\n\n---
**USER'S FINANCIAL DATA:**
{user_data}\n\n---
**FINANCIAL HEALTH REPORT:**"""
final_prompt = ChatPromptTemplate.from_template(final_prompt_template)
analysis_chain = final_prompt | llm | StrOutputParser()
initial_report = analysis_chain.invoke({
    "instructions": FIN_FRIEND_PROMPT_INSTRUCTIONS,
    "context": retrieved_context,
    "user_data": formatted_user_data
})

print("\n\n--- Your Financial Health Report ---\n")
print(initial_report)

# --- Initialize Conversation History for the Chatbot ---
chat_history = f"""Here is the user's original financial data:
{formatted_user_data}
---
Here is the initial financial health report you provided:
{initial_report}
"""

Hello! I'm Fin-Friend... Let's get started!


Great. Now let's break down your monthly expenses.

Thank you for the information. Generating your personalized financial health report...


--- Your Financial Health Report ---

Namaste! It takes real courage and a proactive spirit to take a close look at your finances, and I commend you for taking this important step. As your Fin-Friend, I'm here to guide you with empathy, expertise, and encouragement on your journey to financial mastery. Let's break down your current financial health together.

### Your Financial Health Report

**Understanding Your Current Picture:**

Based on the information you've shared, here's a snapshot of your financial standing:

*   **Monthly Income:** INR 20,000 (Salary)
*   **Monthly Structured Expenses:**
    *   Rent/EMI: INR 3,000
    *   Debt EMI: INR 11,000 (for the INR 2 lakh outstanding debt)
    *   Utilities: INR 0 (which is great if this means they are covered elsewhere or truly negligible)
*   **Tota

In [ ]:
# --- Phase 3: Conversational Cross-Questioning Loop ---

print("\n--- You can now ask follow-up questions about your report ---")

# Define the new prompt template that includes conversation history
follow_up_prompt_template = """
You are Fin-Friend, an expert financial guide. A user has received their initial financial report.
Based on the CHAT HISTORY and the new CONTEXT provided, answer the user's FOLLOW-UP QUESTION.
Your tone should remain supportive and you must still follow your core rules (no direct advice, add disclaimer if needed).

---
CHAT HISTORY:
{chat_history}
---
NEW CONTEXT FROM GUIDE (for the follow-up question):
{context}
---
FOLLOW-UP QUESTION:
{question}
---
ANSWER:
"""
follow_up_prompt = ChatPromptTemplate.from_template(follow_up_prompt_template)

# Create the chain for follow-up questions
follow_up_chain = follow_up_prompt | llm | StrOutputParser()

# Start the interactive loop
while True:
    question = input("\nYour follow-up question (or type 'exit' to quit):\n> ")
    if question.lower() == 'exit':
        break
    if not question.strip():
        continue
    
    # 1. Retrieve new context based on the follow-up question
    new_retrieved_docs = retriever.invoke(question)
    new_context = "\n\n".join([doc.page_content for doc in new_retrieved_docs])
    
    # 2. Invoke the chain with history, new context, and the new question
    answer = follow_up_chain.invoke({
        "chat_history": chat_history,
        "context": new_context,
        "question": question
    })
    
    # 3. Print the answer and update the history
    print("\n--- Fin-Friend's Answer ---")
    print(answer)
    
    # 4. Update the chat history with the latest exchange
    chat_history += f"\n\nUser's Follow-up Question: {question}\nFin-Friend's Answer: {answer}"


--- You can now ask follow-up questions about your report ---

--- Fin-Friend's Answer ---
It's an excellent question to ask, and it shows you're committed to building a stronger financial future! Understanding how much to save is a cornerstone of financial mastery.

Based on the general guidelines shared, particularly for someone likely in their 20s, a common recommendation is to **aim to save 10% of your income**.

Let's look at what this means for your situation:

*   **Your Monthly Income:** INR 20,000
*   **10% Savings Goal:** INR 2,000 per month

**However, let's be realistic about your current financial picture:**

As highlighted in your initial report, after your fixed expenses (Rent/EMI and Debt EMI), you currently have **INR 6,000** remaining for all other essential living costs like groceries, transportation, personal care, and any unexpected expenses. Trying to carve out INR 2,000 for savings from this INR 6,000 while covering all your daily needs would be extremely challe